In [ ]:
# Import libraries and packages for the project 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import csv
from time import sleep
import random
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm #process bar
from underthesea import sentiment #check comment + -

print('- Finish importing packages')


In [ ]:
options = Options()
options.binary_location = "C:\\Program Files\\Google\\Chrome Beta\\Application\\chrome.exe"
driver = webdriver.Chrome(chrome_options = options, executable_path=r'C:\chromedriver_win32\chromedriver.exe')
wait = WebDriverWait(driver,20)

url = "https://shopee.vn/"
shopee_url = 'https://shopee.vn'

# Login to shopee by QR code
login_url = "https://shopee.vn/buyer/login/qr?next=https%3A%2F%2Fshopee.vn%2Fuser%2Fpurchase%2F"
driver.get(login_url)

In [ ]:
brands_url = "https://shopee.vn/mall/brands/"
driver.get(brands_url)
#c1
page_source = BeautifulSoup(driver.page_source)
sections = page_source.find_all('div', {"class":["official-shop-brand-list__section-content"]})
order = 1
count = 0
shop_list = []
headers = ['Order', 'Name', 'URL']

def handle_add_shop(order,name,shop_URL):
    shop_list.append({headers[0]:order,headers[1]:name,headers[2]:shop_URL})
    print("Added new shop: ")
    print("%d - %s - %s"%(order,name,shop_URL))

for section in sections:
    order_section = section.findAll('div',class_="full-brand-list-item")
    if(order==101): # Get enough data
        break
    for shop in order_section:
        cmt_flag = False
        if(order==101):
            break

        # Random order shop
        is_next = random.randint(1,10)
        if(is_next%2==0):
            continue

        shop_URL = shopee_url + shop.find('a', class_="full-brand-list-item__brand-cover-image").get('href') # Get shop url
        name = shop.find('div',class_="full-brand-list-item__brand-name").text # Get shop name
        
        #Click each shop to check quantity > 100 and comment > 20000
        try:
            driver.get(shop_URL)
            print('- Accessing shop: ', shop_URL)
            sleep(2)
            driver.execute_script('window.scrollTo(0, 0);') #scroll to the end of the page
            sleep(1)
        except TimeoutException:
            print("***************************")
            print("Loading took too much time!")
            print("***************************")
            continue

        loop_page_source = BeautifulSoup(driver.page_source,"html.parser")
        items = loop_page_source.find_all('div',{"class":["section-seller-overview__item-text-value"]})
        if(items):
            if(items[0] and items[8]):
                # Check product quantity
                product_quantity = items[0]
                product_quantity = product_quantity.text 

                # Check comment
                comment_quantity = items[8]
                comment_items = comment_quantity.text.split(' ')
                if(len(comment_items)!=4):
                    continue
                comment_quantity = comment_items[1][1:]
                if(not comment_quantity.isdecimal()):
                    tmp = comment_quantity.split(",")
                    cmt_quantity = tmp[0] if len(tmp)>1 else tmp[0][:-1]
                    if(int(cmt_quantity)>=20):
                        cmt_flag = True
                if(cmt_flag):
                    if(not product_quantity.isdecimal()): # Quantity > 100
                        handle_add_shop(order,name,shop_URL)
                        count+=1
                        order+=1  
                    elif(int(product_quantity)>=100): # Check shop product quantity is greater than 100 or not
                        handle_add_shop(order,name,shop_URL)
                        count+=1
                        order+=1    

                sleep(3)

                if(count==10): # 
                    count = 0
                    break
            else:
                continue
            
print("Finished crawling data!")

with open('ShopeeMall.csv', 'w',encoding='utf-8',  newline = '') as file_output:
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shop in shop_list:
        writer.writerow(shop)

print('Mission Completed!')

In [ ]:

headers = ['Order', 'Name', 'URL', "Sales"]
update_data = []
char_dic = {'k':1000,'r': 1000000}

count = 0
sales = 0

for shop in tqdm(df,desc="Processing: "):
    product_list_url = []
    count = 0
    total = 0
    try:
        my_url = shop[2]+"?page=0&sortBy=sales"
        driver.get(my_url)
        print('- Accessing shop: ', shop[2])
        sleep(2)
        
    except:
        print("***************************")
        print("Loading took too much time!")
        print("***************************")
        continue

    print('-- Getting data....')

    for i in range( 3):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
        sleep(0.3)
        loop_page_source = BeautifulSoup(driver.page_source,"html.parser")
        sleep(0.3)
        old_page_source = driver.page_source

        items = loop_page_source.find('div',class_="shop-search-result-view").find_all('a',href=True)

        product_list_url.append(items)
        wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"shopee-icon-button--right"))).click()
        sleep(0.4)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            break
        sleep(2)
    print('-- Finished get data')

    print("--- Access product pages")
    print("************************")
    for page in product_list_url:
        if(count==101):
            break
        for products in page:
            count+=1
            if(count==101):
                break
            price = products.find('span',class_="_3TJGx5").text
            amount_raw = products.find('div',class_="_2R-Crv")

            price = price.replace(".","")

            if(not amount_raw): # ko bán cái nào
                continue
            elif (len(amount_raw.text)==0):
                continue

            tmp = amount = amount_raw.text.split(" ")[2].split("/")[0].replace(",",".")
            
            if tmp[-1].isdecimal():
                amount = float(tmp)
            else:
                if(tmp[-1]=="k"):
                    amount = float(tmp[:-1])*char_dic[tmp[-1]] #1k sp
                else:
                    amount = float(tmp[:-2])*char_dic[tmp[-1]] #1tr sp

            total += float(price)* amount

    update_data.append({headers[0]:shop[0],headers[1]:shop[1],headers[2]:shop[2],headers[3]:total})
    print("************************")

print("Finished crawling data!")


In [ ]:
raw = pd.read_csv('ShopeeMall.csv')
df = np.array(raw)
#c3
headers = ['Order', 'Name', 'URL', "Negative","Positive"]
update_data = []

count = 0
negative = 0
positive = 0
total_cmt = 0
tmp_neg = 0
tmp_pos = 0

for shop in tqdm(df,desc="Processing: "):
    product_list_url = []
    count = 0
    negative = 0
    positive = 0

    try:
        my_url = shop[2]+"?page=0&sortBy=sales"
        driver.get(my_url)
        print('- Accessing shop: ', shop[2])
        sleep(2)
    except:
        print("***************************")
        print("Loading took too much time!")
        print("***************************")
        continue

    print('-- Getting data....')

    for i in range(4): # 1 page 30 product => 4/120 , need 100 product each shop
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to end page
        sleep(0.5)
        loop_page_source = BeautifulSoup(driver.page_source,"html.parser")
        sleep(0.5)
        old_page_source = driver.page_source

        items = loop_page_source.find('div',class_="shop-search-result-view").find_all('a',href=True)

        product_list_url.append(items)
        wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"shopee-icon-button--right"))).click() #qua từng trang sp bằng next
        sleep(0.3)

        new_page_source = driver.page_source
        if new_page_source == old_page_source: #hết qua trang đc 
            break
        sleep(2)
        
    print('-- Finished get data')

    print("--- Access product pages")
    print("************************")
    # need 200 cmt each product
    for page in product_list_url:
        for products in tqdm(page,desc="Current/Total"):
            
            total_cmt = 0
            tmp_neg = 0
            tmp_pos = 0

            count+=1
            if(count==101):
                break
            product_url = url + products.get('href')
            
            driver.get(product_url)
            sleep(2)

            while(1):
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to end page
                sleep(0.5)
                product_detail_page_source = BeautifulSoup(driver.page_source,"html.parser")
                sleep(0.5)
                old_page_source = driver.page_source

                comments = product_detail_page_source.find_all('div', class_="Em3Qhp") # Get comment
                
                if(len(comments)==0): # Section have no comments
                    if total_cmt==0: # First comment section has no comment
                        sleep(2)
                    break

                for cmt in comments:
                    my_text = cmt.text.strip()
                    if(sentiment(my_text)=="negative"):
                        tmp_neg+=1
                    else:
                        tmp_pos+=1

                total_cmt = tmp_neg+tmp_pos

                try:
                    wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"shopee-icon-button--right"))).click() #qua từng trang cmt bằng next
                except TimeoutException:
                    continue
                sleep(0.3)

                new_page_source = driver.page_source
                if new_page_source == old_page_source or total_cmt>=200: #đã qua hết trang hoặc đủ 200 cmt
                    negative+= tmp_neg
                    positive+= tmp_pos
                    break

    update_data.append({headers[0]:shop[0],headers[1]:shop[1],headers[2]:shop[2],headers[3]:negative,headers[4]:positive})
    print("************************")

print("Finished crawling data!")

with open('ShopeeMallSentiment.csv', 'w', encoding='utf-8',  newline = '') as file_output:
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shop in update_data:
        writer.writerow(shop)

print('Mission Completed!')

with open('ShopeeMallSales.csv', 'w',  newline = '') as file_output:
    
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shop in update_data:
        writer.writerow(shop)

print('Mission Completed!')


In [ ]:
#c4
df = pd.read_csv('ShopeeMall.csv')
arraydf = np.array(df)

headers = ['Order', 'Name', 'URL','Type']
result = []

for a in arraydf:
    # access link shop
    driver.get(a[-1])
    sleep(2)
    print('Access shop:' +a[1])

    #access product
    shop_detail = BeautifulSoup(driver.page_source,'html.parser')
    link_firstpd = shop_detail.find_all('a',{'data-sqe': 'link'})
    linkproduct = shopee_url +  link_firstpd[0].get('href')
    driver.get(linkproduct)
    sleep(2)

    #take type
    product_detail = BeautifulSoup(driver.page_source,'html.parser')
    types_shop = product_detail.find_all('a',class_ = 'ClhheV')
    typeshop = types_shop[1].text

    result.append({headers[0]:a[0],headers[1]:a[1],headers[2]:a[2],headers[3]:typeshop})
    
print(result)

with open('ShopeeMallType.csv', 'w', encoding='utf-8', newline = '') as file_output:
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shoptype in result:
        writer.writerow(shoptype)

In [ ]:
#c5
headers = ['Order', 'Name', 'URL', "Rating","Chat Response (%)","Follower"]
char_dic = {'k':1000,'r': 1000000}
update_data = []

for shop in tqdm(df):

    try:
        driver.get(shop[2])
        print('- Accessing shop: ', shop[2])
        sleep(2)
        driver.execute_script('window.scrollTo(0, 0);') #scroll to the end of the page
        sleep(1)
    except TimeoutException:
        print("***************************")
        print("Loading took too much time!")
        print("***************************")
        continue


    loop_page_source = BeautifulSoup(driver.page_source,"html.parser")
    items = loop_page_source.find_all('div',{"class":["section-seller-overview__item-text-value"]})
    
    # Clean data
    if(len(items)==12):
        rating_raw = items[9].text
        chat_response_raw = items[4].text
        follower_raw = items[7].text.replace(",",".")
    else:
        rating_raw = items[11].text
        chat_response_raw = items[4].text
        follower_raw = items[9].text.replace(",",".")

    # Get rating of chat response 
    chat_response = chat_response_raw.split(" ")[0][:-1]

    # Get the number of follower
    unit_char = follower_raw[-1]
    if(unit_char.isdecimal()):
        follower = int(follower_raw[:-1])
    else:
        value = ""
        if(unit_char=="k"):
            value = follower_raw[:-1]
        else:
            value = follower_raw[:-2]
            
        follower = float(value)*char_dic[unit_char]

    # Get rating
    rating = rating_raw.split(" ")[0]


    update_data.append({headers[0]:shop[0],headers[1]:shop[1],headers[2]:shop[2],headers[3]:rating,headers[4]:chat_response,headers[5]:follower})

print("Finished crawling data!")

with open('ShopeeMall5.csv', 'w',  newline = '') as file_output:
    
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shop in update_data:
        writer.writerow(shop)

print('Mission Completed!')


In [ ]:
#c6
df = pd.read_csv('ShopeeMall.csv')
arraydf = np.array(df)

headers = ['Order', 'Name', 'URL','Type','Quantity Discount','Quantity Type Product','Year Join']
result = []

for a in arraydf:
    # access link shop
    driver.get(a[-1])
    sleep(2)
    print('Access shop:' +a[1])
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to end page
    sleep(1)
    shop_detail = BeautifulSoup(driver.page_source,'html.parser')
    sleep(1)
    
    #year
    shop_info = shop_detail.find_all('div',{'class': 'section-seller-overview__item-text-value'})
    shop_info_year = shop_info[-1].text
    year_split = shop_info_year.split(' ')
    if(year_split[1]== 'tháng'):
        yearint = int(year_split[0])
        year = math.floor(yearint/12)
    else: 
        year = year_split[0]

    #count discount
    discount_quantity = shop_detail.find_all('div',{'class': '_7ojJBF'})

    #count type product
    type_product_quantity = shop_detail.find_all('div',{'class': '_1_yYlR'})

    #access product
    link_firstpd = shop_detail.find('a',{'data-sqe': 'link'}).get('href')
    linkproduct = shopee_url +  link_firstpd
    driver.get(linkproduct)
    sleep(2)

    #take type
    product_detail = BeautifulSoup(driver.page_source,'html.parser')
    types_shop = product_detail.find_all('a',class_ = 'ClhheV')
    typeshop = types_shop[1].text
    
    result.append({headers[0]:a[0],
    headers[1]:a[1],
    headers[2]:a[2],
    headers[3]:typeshop,
    headers[4]:len(discount_quantity),
    headers[5]:len(type_product_quantity)-1,
    headers[6]:year})
    
print(result)

with open('ShopeeMallUpdate.csv', 'w', encoding='utf-8', newline = '') as file_output:
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for res in result:
        writer.writerow(res)